In [1]:
############################################################################################################
# Overview: This script cleans the extracted encounter-level data mart.
############################################################################################################

In [2]:
########################################################################################################################
# Import packages
########################################################################################################################
import datetime
import gc
import numpy as np
import os
import pandas as pd
import pyarrow as pa
import warnings
from ast import literal_eval
from pyarrow.parquet import ParquetFile
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

In [3]:
########################################################################################################################
# USER_SPECIFIC SETTING
# DATA_IN_DIR_PATH: Path of the input directory of the encounter-level dataset 
# (created in C01_Data_Transfer.ipynb)
# DATA_OUT_DIR_PATH: Path of the output directory of the encounter-level dataset 
# (created in C01_Data_Transfer.ipynb)
# DICT_IN_FILE_PATH: Path of the input data dictionary file
# (created in C04_Clean_Encounter_Variables.ipynb)
# DICT_OUT_FILE_PATH: Path of the output data dictionary file
########################################################################################################################
DATA_IN_DIR_PATH: str = '../00_Data/00_Raw_Data/'
DATA_OUT_DIR_PATH: str = '../00_Data/01_Cleaned_Data/'
DICT_IN_FILE_PATH: str = '../00_Data/99_Dictionary/Dictionary_v3.xlsx'
DICT_OUT_FILE_PATH: str = '../00_Data/99_Dictionary/Dictionary_v4.xlsx'

In [ ]:
########################################################################################################################
# Load the data dictionary
########################################################################################################################
df_dict_pat: pd.DataFrame = pd.read_excel(DICT_IN_FILE_PATH, sheet_name='Patient')
df_dict_enc: pd.DataFrame = pd.read_excel(DICT_IN_FILE_PATH, sheet_name='Encounter')

In [ ]:
########################################################################################################################
# Load the header of the encounter-level dataset
########################################################################################################################
enc_data_path: str = os.path.join(DATA_IN_DIR_PATH, 'Encounter_full.parquet')
pf: ParquetFile = ParquetFile(enc_data_path)
first_row: pa.lib.RecordBatch = next(pf.iter_batches(batch_size=1))
header: list[str] = pa.Table.from_batches([first_row]).to_pandas().columns.to_list()

In [ ]:
########################################################################################################################
# Initiate a single-column pandas.DataFrame containing the first two columns of the encounter-level dataset
# (i.e., 'EncounterKey' and 'PatientDurableKey')
# Create an extra copy of df_dict_pat for modification
########################################################################################################################
df_out: pd.DataFrame = pd.DataFrame({header[0]: pd.read_parquet(enc_data_path, columns=[header[0]]).iloc[:, 0],
                                     header[1]: pd.read_parquet(enc_data_path, columns=[header[1]]).iloc[:, 0]})
N: int = df_out.shape[0]    
df_dict_enc_v2: pd.DataFrame = pd.DataFrame(None, columns=df_dict_enc.columns)
df_dict_enc_v2.insert(2, 'Sampled_Percentage', np.nan)

In [ ]:
########################################################################################################################
# Part A. Encode and extract data of date & continuous variables
########################################################################################################################
for row_idx, row in df_dict_enc[df_dict_enc['Variable_Type'].isin(['datetime.date',
                                                                  'Date as float',
                                                                  'Date as integer',
                                                                  'Year as float',
                                                                  'Continuous'])].iterrows():

    var_name: str = row['Variable_Name']
    var_type: str = row['Variable_Type']
    remark: str = row['Remark']
    log_head: str = f'[{row_idx}. {var_name}]'

    ####################################################################################################################
    # A1. Load the uncleaned single-column data
    ####################################################################################################################
    df_cur: pd.DataFrame = pd.read_parquet(enc_data_path, columns=[var_name])

    ####################################################################################################################
    # A2. Extract data
    ####################################################################################################################
    # Case 1. Date variables in datetime.date format
    if var_type == 'datetime.date':
        print(f'{log_head}datetime.date -> Date')
        df_cur[var_name] = pd.to_datetime(df_cur[var_name])
        new_var_type: str = 'Date'

    # Case 2. Date variables in float format
    elif var_type in ['Date as float', 'Date as integer']:
        print(f'{log_head}Date as integer/float -> Date')
        df_cur[var_name] = df_cur[var_name].apply(lambda x: str(int(x)) if pd.notna(x) else None)
        df_cur[var_name] = pd.to_datetime(df_cur[var_name], format='%Y%m%d', errors='coerce')
        new_var_type: str = 'Date'

    # Case 3. Year variables in float format
    elif var_type in ['Year as float']:
        print(f'{log_head}Year as float -> Year')
        df_cur[var_name] = df_cur[var_name].apply(lambda x: str(int(x)) if pd.notna(x) else None)
        df_cur[var_name] = pd.to_datetime(df_cur[var_name], format='%Y', errors='coerce')
        new_var_type: str = 'Year'

    # Case 4. Continuous variables
    else:
        print(f'{log_head}Continuous (as is)')
        new_var_type: str = 'Continuous'

    df_out[var_name] = df_cur[var_name]

    ####################################################################################################################
    # A3. Update dictionary
    ####################################################################################################################
    sampled_ratio: int = int(round(df_cur[var_name].notna().sum() / N, 2) * 100)
    record_cur: pd.DataFrame = pd.DataFrame({'Variable_Name': [var_name],
                                             'Sample_Size': int(df_cur[var_name].notna().sum()),
                                             'Sampled_Percentage': [sampled_ratio],
                                             'Encoded_Values': [np.nan],
                                             'Variable_Type': [new_var_type],
                                             'Remark': [remark]})
    df_dict_enc_v2 = pd.concat([df_dict_enc_v2, record_cur], ignore_index=True)
    del df_cur
    gc.collect()

In [ ]:
########################################################################################################################
# Part B. Encode all binary / ordinal variables (no nominal variables found)
########################################################################################################################
df_dict_enc_sub: pd.DataFrame = df_dict_enc[df_dict_enc['Variable_Type'].isin(['Binary', 'Ordinal'])]
df_out_cat: pd.DataFrame = pd.DataFrame(None)        # Create empty pandas.DataFrame for the data and dictionary of
df_dict_enc_cat: pd.DataFrame = pd.DataFrame(None)   # the categorical variables

for row_idx, row in df_dict_enc_sub.iterrows():
    var_name: str = row['Variable_Name']
    var_type: str = row['Variable_Type']
    encoder: dict[str, int] = literal_eval(row['Encoded_Values'])
    remark: str = row['Remark']
    log_head: str = f'[{row_idx}. {var_name}]'

    ####################################################################################################################
    # B1. Load the uncleaned single-column data
    ####################################################################################################################
    df_cur: pd.DataFrame = pd.read_parquet(enc_data_path, columns=[var_name])

    ####################################################################################################################
    # B2. Identify the codes for missingness
    ####################################################################################################################
    missing_encoder: dict[str, int] = {k: v for k, v in encoder.items() if v is not None and v < 0}
    non_missing_encoder: dict[str, int] = {k: v for k, v in encoder.items() if v is not None and v >= 0}
    missing_decoder: dict[int, str] = {v: k for k, v in missing_encoder.items()}
    non_missing_decoder: dict[int, str] = {v: k for k, v in non_missing_encoder.items()}
    assert len(missing_decoder) == 0   # We do not have any missing values encoded in the encounter-level dataset

    ####################################################################################################################
    # B3. Apply the encoder to the data
    ####################################################################################################################
    df_cur[var_name].replace(encoder, inplace=True)

    #####################################################################################################################
    # B4. Handle non-ordinal variables first
    #####################################################################################################################
    if var_type == 'Binary':
        assert len(non_missing_encoder) == 2
        assert set(non_missing_encoder.values()) == {0, 1}, non_missing_encoder.values()
    
        # 4.1 Encode only the positive value for binary variables, and all values for nominal variables
        for k, v in non_missing_decoder.items():
            if var_type == 'Binary' and k == 0:
                continue
    
            new_var_name: str = f'{var_name}^{k}={v}'
            df_out_cat[new_var_name] = df_cur[var_name].apply(
                lambda x: np.nan if pd.isna(x) or x < 0
                else int(x == k)
            ).astype('Int32')   # Cleaned data
    
            sample_size: int = df_out_cat[new_var_name].notna().sum()
            sampled_ratio: int = int(round(sample_size / N, 2) * 100)
            record_cur: pd.DataFrame = pd.DataFrame({'Variable_Name': [new_var_name],
                                                     'Sample_Size': [sample_size],
                                                     'Sampled_Percentage': [sampled_ratio],
                                                     'Encoded_Values': [{0: 'No', 1: 'Yes'}],
                                                     'Variable_Type': ['Binary'],
                                                     'Remark': [remark]})
    
            df_dict_enc_cat = pd.concat([df_dict_enc_cat, record_cur], ignore_index=True)   # Cleaned dictionary
            print(f'{log_head}{(k, v)} encoded.')
    
    ########################################################################################################################
    # B5. Handle nominal encoding
    ########################################################################################################################
    else:
        assert len(non_missing_encoder) > 2

        # 5.1 Encode only the non-negative values
        df_out_cat[var_name] = df_cur[var_name].apply(
            lambda x: np.nan if pd.isna(x) or x < 0 else x
        ).astype('Int32')
    
        sample_size: int = df_out_cat[var_name].notna().sum()
        sampled_ratio: int = int(round(sample_size / N, 2) * 100)
        record_cur: pd.DataFrame = pd.DataFrame({'Variable_Name': [var_name],
                                                 'Sample_Size': [sample_size],
                                                 'Sampled_Percentage': [sampled_ratio],
                                                 'Encoded_Values': [non_missing_decoder],
                                                 'Variable_Type': ['Ordinal'],
                                                 'Remark': [remark]})
    
        df_dict_enc_cat = pd.concat([df_dict_enc_cat, record_cur], ignore_index=True)   # Cleaned dictionary
        print(f'{log_head}Encoded.')
    del df_cur
    gc.collect()

In [ ]:
########################################################################################################################
# Part C. Handle the ICD-10 encoding of OutpatPrimaryDx, OutpatNonPrimaryDx, EDPrimDx
########################################################################################################################

########################################################################################################################
# C1. Extract the three columns of data from the encounter-level dataset
########################################################################################################################
icd_cols: list[str] = ['OutpatPrimaryDx', 'OutpatNonPrimaryDx', 'EDPrimDx']
df: pd.DataFrame = pd.read_parquet(enc_data_path, columns=icd_cols)

########################################################################################################################
# C2. Turn the values of each column into a list of ICD codes
########################################################################################################################
unique_codes: list[str] = []
for col in df:
    df[col] = df[col].apply(lambda x: [] if pd.isna(x) else x.split(','))
    unique_codes += [icd_code for code_list in df[col].values for icd_code in code_list]

unique_codes = sorted(set(unique_codes))
print(f'{len(unique_codes)} unique ICD codes found.')

In [ ]:
########################################################################################################################
# C3. Write a function to parse an ICD code into its chapter-level code
# See https://icd.who.int/browse10/2019/en and https://icd10data.com/ICD10CM/Codes
########################################################################################################################

# Define the 22 chapters
chapter_dict: dict[int, list[str]] = {
    1:  [f'{char}{str(i).zfill(2)}' for char in ['A', 'B'] for i in range(100)],
    2:  [f'C{str(i).zfill(2)}' for i in range(98)] + [f'D{str(i).zfill(2)}' for i in range(50)] + ['C4A', 'C7A', 'C7B', 'D3A'],
    3:  [f'D{str(i).zfill(2)}' for i in range(50, 90)],
    4:  [f'E{str(i).zfill(2)}' for i in range(100)],
    5:  [f'F{str(i).zfill(2)}' for i in range(100)],
    6:  [f'G{str(i).zfill(2)}' for i in range(100)],
    7:  [f'H{str(i).zfill(2)}' for i in range(60)],
    8:  [f'H{str(i).zfill(2)}' for i in range(60, 96)],
    9:  [f'I{str(i).zfill(2)}' for i in range(100)] + ['I1A', 'I5A'],
    10: [f'J{str(i).zfill(2)}' for i in range(100)],
    11: [f'K{str(i).zfill(2)}' for i in range(96)],
    12: [f'L{str(i).zfill(2)}' for i in range(100)],
    13: [f'M{str(i).zfill(2)}' for i in range(100)] + ['M1A'],
    14: [f'N{str(i).zfill(2)}' for i in range(100)],
    15: [f'O{str(i).zfill(2)}' for i in range(100)] + ['O9A'],
    16: [f'P{str(i).zfill(2)}' for i in range(97)],
    17: [f'Q{str(i).zfill(2)}' for i in range(100)],
    18: [f'R{str(i).zfill(2)}' for i in range(100)],
    19: [f'S{str(i).zfill(2)}' for i in range(100)] + [f'T{str(i).zfill(2)}' for i in range(99)],
    20: [f'{char}{str(i).zfill(2)}' for char in ['V', 'W', 'X', 'Y'] for i in range(100)],
    21: [f'Z{str(i).zfill(2)}' for i in range(100)] + ['Z3A'],
    22: [f'U{str(i).zfill(2)}' for i in range(86)]
}

chapter_dict = {k: set(v) for k, v in chapter_dict.items()}  # Set-operation is much faster than list-operation in Python

def icd_chapter(icd_code):
    parent_code = icd_code.split('.')[0] if '.' in icd_code else icd_code
    for k, v in chapter_dict.items():
        if parent_code in v:
            return k
    return np.nan

In [ ]:
########################################################################################################################
# C4. Create a copy of df and apply the function above from df to the new copy
########################################################################################################################
df_c: pd.DataFrame = pd.DataFrame(None)
for col in icd_cols:
    df_c[col] = df[col].apply(lambda x: [icd_chapter(y) for y in x if not y.startswith('IMO')])
    print(f'Finished encoding ICD chapter codes for {col}.')

In [ ]:
########################################################################################################################
# C5. One-hot encoding
########################################################################################################################
for col in icd_cols:
    for ch_idx in range(1, 23, 1):
        new_var_name: str = f'{col}^ch{ch_idx}'
        df_c[new_var_name] = df_c[col].apply(
            lambda x: np.nan if len(x) == 0 else int(ch_idx in x)
        ).astype('Int32')
        print(f'Finished encoding chapter {ch_idx} for {col}')

In [ ]:
########################################################################################################################
# C6. Concatenate the data and update the data dictionary
########################################################################################################################
df_dict_enc_v2 = pd.concat([df_dict_enc_v2, df_dict_enc_cat], axis=0)
df_out = pd.concat([df_out, df_out_cat], axis=1)

df_c.drop(columns=icd_cols, inplace=True)
for col in df_c.columns:
    sample_size: int = df_c[col].notna().sum()
    sampled_ratio: int = int(round(sample_size / N, 2) * 100)
    record_cur: pd.DataFrame = pd.DataFrame({'Variable_Name': [col],
                                             'Sample_Size': [sample_size],
                                             'Sampled_Percentage': [sampled_ratio],
                                             'Encoded_Values': [{0: 'No', 1: 'Yes'}],
                                             'Variable_Type': ['Binary'],
                                             'Remark': [np.nan]})
    df_dict_enc_v2 = pd.concat([df_dict_enc_v2, record_cur], ignore_index=True)  # Cleaned dictionary
df_out = pd.concat([df_out, df_c], axis=1)

In [ ]:
########################################################################################################################
# Part D. Concatenate the data sets and dictionaries respectively, and save them
########################################################################################################################
os.makedirs(DATA_OUT_DIR_PATH, exist_ok=True)
data_file_path: str = os.path.join(DATA_OUT_DIR_PATH, 'Encounter_full_v1.parquet')
df_out.to_parquet(data_file_path)
print(f'Cleaned data (v1) saved with dimension={df_out.shape}')

with pd.ExcelWriter(DICT_OUT_FILE_PATH) as writer:
    df_dict_pat.to_excel(writer, sheet_name='Patient', index=False)
    df_dict_enc_v2.to_excel(writer, sheet_name='Encounter', index=False)
print(f'Encounter-level dictionary updated with {df_dict_enc_v2.shape[0]} variables.')

# The data dictionary should contain 2 variables less than the data.